In [1]:
# pip install yfinance autogluon gradio

  Using cached torch-2.3.1-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)


  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached triton-2.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.4 kB)


Using cached torch-2.3.1-cp310-cp310-manylinux1_x86_64.whl (779.1 MB)
Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
Using cached triton-2.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (168.1 MB)


  Attempting uninstall: triton
    Found existing installation: triton 3.0.0
    Uninstalling triton-3.0.0:
      Successfully uninstalled triton-3.0.0
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 9.1.0.70
    Uninstalling nvidia-cudnn-cu12-9.1.0.70:
      Successfully uninstalled nvidia-cudnn-cu12-9.1.0.70
  Attempting uninstall: torch
    Found existing installation: torch 2.4.1
    Uninstalling torch-2.4.1:
      Successfully uninstalled torch-2.4.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# 주식 종목 리스트
tickers = ["AAPL", "MSFT", "TSLA", "NVDA"]
start_date = "2014-01-02"

# 현재 날짜 전날을 end_date로 설정
end_date = (datetime.now() - timedelta(1)).strftime('%Y-%m-%d')

# 주식 데이터를 각 티커별로 불러오기
stock_data = {}
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    data = data[['Close']].reset_index()
    data.columns = ['date', ticker]
    stock_data[ticker] = data

# 각 종목의 데이터를 하나의 데이터프레임으로 병합
merged_data = stock_data['AAPL']
for ticker in tickers[1:]:
    merged_data = pd.merge(merged_data, stock_data[ticker], on='date', how='inner')

# 데이터 확인
merged_data.tail()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,date,AAPL,MSFT,TSLA,NVDA
2710,2024-10-09,229.539993,417.459991,241.050003,132.649994
2711,2024-10-10,229.039993,415.839996,238.770004,134.809998
2712,2024-10-11,227.550003,416.320007,217.800003,134.800003
2713,2024-10-14,231.300003,419.140015,219.160004,138.070007
2714,2024-10-15,233.850006,418.739990,219.570007,131.600006


In [3]:
# 데이터프레임을 CSV 파일로 저장
merged_data.to_csv('stock_data.csv', index=False)

In [4]:
merged_data.tail()

,date,AAPL,MSFT,TSLA,NVDA
2710,2024-10-09,229.539993,417.459991,241.050003,132.649994
2711,2024-10-10,229.039993,415.839996,238.770004,134.809998
2712,2024-10-11,227.550003,416.320007,217.800003,134.800003
2713,2024-10-14,231.300003,419.140015,219.160004,138.070007
2714,2024-10-15,233.850006,418.739990,219.570007,131.600006


In [5]:
print(merged_data.columns)

Index(['date', 'AAPL', 'MSFT', 'TSLA', 'NVDA'], dtype='object')


In [6]:
df = pd.read_csv("./stock_data.csv")
df.head()

,date,AAPL,MSFT,TSLA,NVDA
0,2014-01-02,19.754642,37.160000,10.006667,0.39650
1,2014-01-03,19.320715,36.910000,9.970667,0.39175
2,2014-01-06,19.426071,36.130001,9.800000,0.39700
3,2014-01-07,19.287144,36.410000,9.957333,0.40350
4,2014-01-08,19.409286,35.759998,10.085333,0.40900


In [7]:
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
import pandas as pd

# 데이터 구성
train_df = pd.concat([
    pd.DataFrame({
        'item_id': [ticker] * len(df),
        'timestamp': df['date'],
        'target': df[ticker]
    })
    for ticker in ["AAPL", "MSFT", "TSLA", "NVDA"]
])

# TimeSeriesDataFrame으로 변환
train_df = TimeSeriesDataFrame.from_data_frame(
    train_df,
    id_column='item_id',
    timestamp_column='timestamp'
)



/home/user/miniforge3/envs/automl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
train_df

target
item_id timestamp             
AAPL    2014-01-02   19.754642
        2014-01-03   19.320715
        2014-01-06   19.426071
        2014-01-07   19.287144
        2014-01-08   19.409286
...                        ...
NVDA    2024-10-09  132.649994
        2024-10-10  134.809998
        2024-10-11  134.800003
        2024-10-14  138.070007
        2024-10-15  131.600006

[10860 rows x 1 columns]

In [10]:
# TimeSeriesPredictor 초기화
predictor = TimeSeriesPredictor(
    prediction_length=30, 
    path="autogluon-stock-prices",
    target="target",
    eval_metric="MASE",
    freq='D'  # 일일 단위 주기 설정
)

# 모델 학습
predictor.fit(
    train_df,  # 위 데이터가 TimeSeriesDataFrame 형태로 변환된 상태여야 함
    presets="medium_quality",
    time_limit=600
)

# 예측
predictions = predictor.predict(train_df)

# 예측 결과 확인
print(predictions.head())

Beginning AutoGluon training... Time limit = 600s
AutoGluon will save models to 'autogluon-stock-prices'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Mar 29 23:14:13 UTC 2024
CPU Count:          12
GPU Count:          0
Memory Avail:       11.21 GB / 15.31 GB (73.2%)
Disk Space Avail:   41.02 GB / 237.85 GB (17.2%)
Setting presets to: medium_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MASE,
 'freq': 'D',
 'hyperparameters': 'light',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 30,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'time_limit': 600,
 'verbosity': 2}

train_data with frequency 'None' has been resampled to frequency 'D'.
Provide

                          mean         0.1         0.2         0.3  \
item_id timestamp                                                    
AAPL    2024-10-16  232.462406  230.094457  230.960993  231.665244   
        2024-10-17  232.449407  229.749823  230.794888  231.595818   
        2024-10-18  232.336282  229.337794  230.508072  231.391170   
        2024-10-19  232.343301  229.050272  230.340637  231.303005   
        2024-10-20  232.340371  228.767420  230.167755  231.214458   

                           0.4         0.5         0.6         0.7  \
item_id timestamp                                                    
AAPL    2024-10-16  232.148789  232.462406  233.217886  233.405839   
        2024-10-17  232.051562  232.449407  233.353905  233.535021   
        2024-10-18  231.863836  232.336282  233.337398  233.541220   
        2024-10-19  231.805914  232.343301  233.442058  233.685034   
        2024-10-20  231.747870  232.340371  233.531917  233.825277   

                  

In [11]:
import gradio as gr

# 예측 결과를 표시하는 함수 (단일 티커)
def display_predictions(ticker):
    if ticker in predictions.index.levels[0]:
        prediction_df = predictions.loc[ticker]['mean'].reset_index()
        return prediction_df
    else:
        return f"{ticker}는 예측 결과에 없습니다."

# Gradio 인터페이스 생성 (티커 선택)
interface = gr.Interface(
    fn=display_predictions,
    inputs=gr.Dropdown(choices=["AAPL", "MSFT", "TSLA", "NVDA"], label="Select Ticker"),
    outputs="dataframe",
    title="30일 주식 종가 예측",
    description="AutoGluon을 사용한 선택한 주식의 30일 종가 예측"
)

# 앱 실행
interface.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Created dataset file at: .gradio/flagged/dataset1.csv


Traceback (most recent call last):
  File "/home/user/miniforge3/envs/automl/lib/python3.10/site-packages/gradio/queueing.py", line 622, in process_events
    response = await route_utils.call_process_api(
  File "/home/user/miniforge3/envs/automl/lib/python3.10/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/user/miniforge3/envs/automl/lib/python3.10/site-packages/gradio/blocks.py", line 2024, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
  File "/home/user/miniforge3/envs/automl/lib/python3.10/site-packages/gradio/blocks.py", line 1830, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
  File "/home/user/miniforge3/envs/automl/lib/python3.10/site-packages/gradio/components/dataframe.py", line 259, in postprocess
    value = pd.read_csv(value)  # type: ignore
  File "/home/user/miniforge3/envs/automl/lib/python3.10/site-packages

In [16]:
import gradio as gr
import matplotlib.pyplot as plt

# 예측 결과를 시각화하는 함수 (2014-01-02부터 시작)
def display_predictions(ticker):
    if ticker in predictions.index.levels[0]:
        prediction_df = predictions.loc[ticker]['mean'].reset_index()
        
        # 실제 종가 데이터 (2014-01-02부터)
        historical_df = train_df.loc[ticker].reset_index()

        # 시각화
        plt.figure(figsize=(10, 6))
        plt.plot(historical_df['timestamp'], historical_df['target'], label=f"{ticker} 실제 종가", color='blue')
        plt.plot(prediction_df['timestamp'], prediction_df['mean'], label=f"{ticker} 예측 종가", color='orange')
        plt.fill_between(prediction_df['timestamp'], predictions.loc[ticker]['0.1'], predictions.loc[ticker]['0.9'], color='gray', alpha=0.3, label='90% 예측 구간')
        plt.xlabel('날짜')
        plt.ylabel('종가')
        plt.title(f"{ticker}의 2014-01-02부터 30일 예측 종가")
        plt.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()

        # 그래프를 이미지로 반환
        plt.savefig("prediction_plot.png")
        return "prediction_plot.png"
    else:
        return f"{ticker}는 예측 결과에 없습니다."

    
    
    
# Gradio 인터페이스 생성 (티커 선택)
interface = gr.Interface(
    fn=display_predictions,
    inputs=gr.Dropdown(choices=["AAPL", "MSFT", "TSLA", "NVDA"], label="Select Ticker"),
    outputs="image",
    title="2014-01-02부터 30일 주식 종가 예측",
    description="AutoGluon을 사용한 선택한 주식의 2014-01-02부터 30일 종가 예측 및 시각화"
)

# 앱 실행
interface.launch()


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


/tmp/ipykernel_76242/519789105.py:22: UserWarning: Glyph 45216 (\N{HANGUL SYLLABLE NAL}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_76242/519789105.py:22: UserWarning: Glyph 51676 (\N{HANGUL SYLLABLE JJA}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_76242/519789105.py:22: UserWarning: Glyph 51333 (\N{HANGUL SYLLABLE JONG}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_76242/519789105.py:22: UserWarning: Glyph 44032 (\N{HANGUL SYLLABLE GA}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_76242/519789105.py:22: UserWarning: Glyph 51032 (\N{HANGUL SYLLABLE YI}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_76242/519789105.py:22: UserWarning: Glyph 48512 (\N{HANGUL SYLLABLE BU}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_76242/519789105.py:22: UserWarning: Glyph 53552 (\N{HANGUL SYLLABLE TEO}) missing from font(s) DejaVu Sans.
  plt.tight_lay

In [18]:
# import gradio as gr
# import matplotlib.pyplot as plt

# # 예측 결과를 시각화하는 함수 (지난 30일간의 실제 주가와 예측 주가 나누어 표시)
# def display_predictions(ticker):
#     if ticker in predictions.index.levels[0]:
#         prediction_df = predictions.loc[ticker]['mean'].reset_index()

#         # 2014-01-02부터의 실제 종가 데이터
#         historical_df = train_df.loc[ticker].reset_index()

#         # 지난 30일간의 실제 종가 데이터
#         last_30_days = historical_df.tail(30)

#         # 첫 번째 그래프: 지난 30일 실제 종가
#         plt.figure(figsize=(10, 6))
#         plt.plot(last_30_days['timestamp'], last_30_days['target'], label=f"{ticker} 지난 30일 실제 종가", color='green')
#         plt.xlabel('날짜')
#         plt.ylabel('종가')
#         plt.title(f"{ticker}의 지난 30일 실제 종가")
#         plt.legend()
#         plt.xticks(rotation=45)
#         plt.tight_layout()
#         plt.savefig("last_30_days_plot.png")
#         plt.close()

#         # 두 번째 그래프: 30일 예측 종가
#         plt.figure(figsize=(10, 6))
#         plt.plot(prediction_df['timestamp'], prediction_df['mean'], label=f"{ticker} 예측 종가", color='orange')
#         plt.fill_between(prediction_df['timestamp'], predictions.loc[ticker]['0.1'], predictions.loc[ticker]['0.9'], color='gray', alpha=0.3, label='90% 예측 구간')
#         plt.xlabel('날짜')
#         plt.ylabel('종가')
#         plt.title(f"{ticker}의 30일 예측 종가")
#         plt.legend()
#         plt.xticks(rotation=45)
#         plt.tight_layout()
#         plt.savefig("prediction_plot.png")
#         plt.close()

#         return ["last_30_days_plot.png", "prediction_plot.png"]
#     else:
#         return f"{ticker}는 예측 결과에 없습니다."

# # Gradio 인터페이스 생성 (티커 선택)
# interface = gr.Interface(
#     fn=display_predictions,
#     inputs=gr.Dropdown(choices=["AAPL", "MSFT", "TSLA", "NVDA"], label="Select Ticker"),
#     outputs=[gr.Image(type="filepath"), gr.Image(type="filepath")],  # 두 개의 그래프를 하나의 버튼으로 출력
#     title="지난 30일 실제 주가 및 30일 예측 주가",
#     description="AutoGluon을 사용한 선택한 주식의 30일 예측 및 지난 30일 주가 시각화"
# )

# # 앱 실행
# interface.launch()


* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/user/miniforge3/envs/automl/lib/python3.10/site-packages/gradio/queueing.py", line 622, in process_events
    response = await route_utils.call_process_api(
  File "/home/user/miniforge3/envs/automl/lib/python3.10/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/user/miniforge3/envs/automl/lib/python3.10/site-packages/gradio/blocks.py", line 2024, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
  File "/home/user/miniforge3/envs/automl/lib/python3.10/site-packages/gradio/blocks.py", line 1780, in postprocess_data
    self.validate_outputs(block_fn, predictions)  # type: ignore
  File "/home/user/miniforge3/envs/automl/lib/python3.10/site-packages/gradio/blocks.py", line 1739, in validate_outputs
    raise ValueError(
ValueError: A  function (display_predictions) didn't return enough output values (needed: 2, retu